# Introduction
In notebook 47 I added the vitamin B12 requirement into the biomass reaction. However I noticed that we should revisit the vitamin B12 biosynthesis as it doesnt seem to match comments made about it in the thesis. 

I opened issue #92 to discuss how to incoporate this into the model. Based on information in the thesis, provided by martyn and found in literature (e.g. Moore et al. 2012, doi:10.1042/BST20120066) we've come to the conclusion that our strain likely uses a hybrid (oxygen-independent) mechanism. The upstream part of the pathway comes from the anaerobic pathway. At the step of precorrin-5, E.C. 4.99.1.3 is used to incorporate the cobalt ion into the molecule. Then the rest of the pathway follows the aerobic part to yield cobyrinate.

Here I will make sure that these parts are incorporated properly in the model, and provide some rational for some decisions made with specific steps.

I will work top-down, starting from precorrin 2.

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [2]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
model_e_coli = cameo.load_model ('iML1515')

PC20M: from precorrin 2 (shcl_c) to Precorrin 3A via the CobI protein.

The next step should convert precorrin 3A to Precorrin 4. In the classical aerobic pathway, this is done via CobG and CobJ, however it seems these enzymes are not present in our strain. These two enzymes are resposible for the oxygen dependent ring contraction. 

Alternatively, our strain does have the CbiH enzyme annotated. This is the anaerobic equivalent that causes ring-contraction. It is also hypothesized that the CbiH cannot discriminate between cobalt-factor III and cobalt-precorrin-3. So it may be that the CbiH enzyme in our strain can perform this conversion without the necessity of oxygen. This enzyme is generally understudied and its function not well understood, and so this conversion will be added.

At the same time, as checking the PRE3AOR (CobG) and PC17M (CobJ) reactions, they do not seem to be present in our strain. Therefore to prevent them from carrying incorrect flux, they should be removed. 

Note: with this hypothesis, the structure of the precorrin-4 will be slightly different than would be traditionally considered, so i will name it precorrin4a. just to indicate it is a bit different. This difference gets corrected with the cobalt gets added to the molecule. (i.e. Co-precorrin5A is formed)

In [4]:
model.metabolites.pre4_c.id = 'pre4a_c'

In [5]:
model.metabolites.pre4a_c.formula = 'C44H54N4O16'

In [6]:
model.metabolites.pre4a_c.notes['NOTES'] = 'Slightly different molecule as a result of hybrid pathway'

In [7]:
del model.metabolites.pre4a_c.annotation['chebi']
del model.metabolites.pre4a_c.annotation['kegg.compound']
del model.metabolites.pre4a_c.annotation['metanetx.chemical']

In [8]:
#add reaction
model.add_reaction(Reaction(id='PRE3MT', name = 'cobalt-factor III methyltransferase'))

In [9]:
model.reactions.PRE3MT.annotation['sbo']= 'SBO:0000176'
model.reactions.PRE3MT.annotation['kegg.reaction']= 'R11580'
model.reactions.PRE3MT.annotation['ec-code']= '2.1.1.272'

In [10]:
model.groups.get_by_id('00860 - Porphyrin and chlorophyll metabolism').add_members(model.reactions.PRE3MT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



the reaction has an unknown electron donor, which should be added. This should also be regenerated. From physioloigcal reasoning, I would expect this to be NADPH, and so that will be coupled here.

In [11]:
#add new reduced acceptor
model.add_metabolites(Metabolite(id='hacc4_c', name = 'Ambiguous electron acceptor', compartment = 'c', charge =-1, formula = 'HR'))

In [12]:
model.metabolites.hacc4_c.annotation = model.metabolites.hacc_c.annotation

In [13]:
#add oxidized electron acceptor
model.add_metabolites(Metabolite(id='acc4_c', name = 'Ambiguous electron acceptor', compartment = 'c', charge =0, formula = 'R'))

In [14]:
model.metabolites.acc4_c.annotation = model.metabolites.acc_c.annotation

In [15]:
model.reactions.PRE3MT.add_metabolites({
    model.metabolites.amet_c:-1,
    model.metabolites.pre3a_c:-1,
    model.metabolites.hacc4_c:-1,
    model.metabolites.ahcys_c:1,
    model.metabolites.pre4a_c:1,
    model.metabolites.acc4_c: 1,
})

In [16]:
#Add regeneration of acceptor with NADPH
model.add_reaction(Reaction(id='ACCR4', name = 'acceptor regeneration variant 4'))

In [17]:
model.reactions.ACCR4.annotation['sbo'] = 'SBO:0000176'

In [18]:
model.reactions.ACCR4.notes['NOTES'] = 'Assumed regeneration with NADPH'

In [19]:
model.reactions.ACCR4.bounds = (-1000,1000)

In [20]:
model.reactions.ACCR4.add_metabolites({
    model.metabolites.acc4_c:-1,
    model.metabolites.hacc4_c:1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1
})

In [21]:
#remove the two reactions which should not be present

In [22]:
model.remove_reactions(model.reactions.PRE3AOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [23]:
model.remove_reactions(model.reactions.PC17M)

Now that we have made the slightly modified precorrin4'A', this can be converted into precorrin 5, via the cobM enzyme. Here again, due to the ring cyclization in the precorrin 4'A', I'll change the precorrin 5 metabolite to precorrin5'A' aswell, and add a note to this metabolite so people are aware.

The reaction is the PC11M reaction, which is still correct and can be left as is.

In [24]:
model.metabolites.pre5_c.id = 'pre5a_c'

In [25]:
model.metabolites.pre5a_c.notes['NOTES'] =  'Slightly different molecule as a result of hybrid pathway'

In [26]:
del model.metabolites.pre5a_c.annotation['chebi']
del model.metabolites.pre5a_c.annotation['kegg.compound']
del model.metabolites.pre5a_c.annotation['metanetx.chemical']

In [27]:
model.metabolites.pre5a_c.formula = 'C45H56N4O16'

We've now finished the 'aerobic' branch, and so I will commit this into the repo before continuing here.

In [28]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

The next step in the proposed hybrid pathway would be the integration of cobalt into the molecule. Beata proposes that this happens in the similar fashion as in B. megatrium and S. enterica: with a sirohydrochlorin cobaltochelatase (EC 4.99.1.3). This would convert the Precorrin5'A' into cobalt-precorrin5A. 

I will add this reaction into the model.

Also, in the model currently, the reaction carries by CobF is present. However, there is no gene annotation for CobF. To prevent the normal aerobic pathway from running, I will remove the P6AS reaction that is catalyzed by CobF from the model too.

In [ ]:
model.remove_reactions(model.reactions.P6AS)

In [ ]:
#save aerobic part

In [181]:
model.reactions.PC11M.check_mass_balance()

{}

In [ ]:
should be this reaction:PC11M
    just check that it is mass balanced!

In [174]:
model.metabolites.pre5a_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,PC11M,100.0,0.000156,amet_c + pre4a_c --> ahcys_c + h_c + pre5a_c
CONSUMING,P6AS,100.0,0.000156,amet_c + h2o_c + pre5a_c --> ac_c + ahcys_c + ...


In [ ]:
remove P6AS: this is the CobF gene, which isnt annotated

In [175]:
model.metabolites.pre4a_c

Metabolite identifier,pre4a_c
Name,Precorrin 4
Memory address,0x0190e3bfd908
Formula,C44H54N4O16
Compartment,c
In 2 reaction(s),"PC11M, PRE3MT"


#need to check that the model actually uses this pathway
so delete steps from the pathways that are not annotated and or used! 
    
- e.g. delete cobF as it isnt annotated to prevent complete aerobic pathway
remove CbiT and CbiE